Simple Next Word Prediction Using Transformers and Pytourch
---





In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 5.5 MB/s 
     |████████████████████████████████| 9.1 MB 8.7 MB/s 
     |████████████████████████████████| 4.4 MB 40.7 MB/s 
     |████████████████████████████████| 1.2 MB 30.6 MB/s 
     |████████████████████████████████| 362 kB 43.4 MB/s 
     |████████████████████████████████| 6.6 MB 39.0 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 1.8 MB 48.3 MB/s 
     |████████████████████████████████| 101 kB 11.0 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 145 kB 57.2 MB/s 
     |████████████████████████████████| 181 kB 72.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 212 kB 60.1 MB/s 
     |████████████████████████████████| 1.1 MB 52.1 MB/s 
     |███████████████████████████████

In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



**Import Libraries**

---



In [4]:
import os
import torch
from torch.nn import functional as F
import string
from transformers import AutoModelWithLMHead, AutoTokenizer, top_k_top_p_filtering, logging
logging.set_verbosity_error()

**variables Declaration**

---



In [12]:
no_words_to_be_predicted = globals()
enter_input_text = globals()

**Function To get model configurations from users**

---



In [13]:

def set_model_config(**kwargs):
  for key, value in kwargs.items():
    print("{0} = {1}".format(key, value))
  
  no_words_to_be_predicted = list(kwargs.values())[0] # integer values
  enter_input_text = list(kwargs.values())[1] #only string

  return no_words_to_be_predicted,enter_input_text

**Load Model and Tokenizer**

---



In [14]:
def load_model():
  try:
      gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
      gpt_model = AutoModelWithLMHead.from_pretrained("gpt2")
      return gpt_tokenizer,gpt_model
  except Exception as e:
    pass

**Functions to encode & decode input text** 

---



In [15]:
# gpt encode
def encode_gpt(tokenizer, text_sentence, add_special_tokens=False):
  input_ids = tokenizer.encode(text_sentence, return_tensors="pt")
  return input_ids

# gpt decode
def decode_gpt(tokenizer, input_ids, pred, top_clean):
  filtered_next_token_logits = top_k_top_p_filtering(pred, top_k=top_clean, top_p=1.0)

  # sample
  probs = F.softmax(filtered_next_token_logits, dim=-1)
  next_token = torch.multinomial(probs, num_samples=top_clean)
  generated = torch.cat([input_ids, next_token], dim=-1)  
  resulting_string = tokenizer.decode(generated.tolist()[0])
  return resulting_string

**Wrapper function for encoder and decoder**

---



In [16]:
def get_all_predictions(text_sentence, top_clean=5):
    input_ids = encode_gpt(gpt_tokenizer, text_sentence)
    with torch.no_grad():
     predict = gpt_model(input_ids)[0][:, -1, :]
    gpt = decode_gpt(gpt_tokenizer, input_ids, predict, top_clean)
    return {'gpt': gpt}

In [17]:
def get_prediction_end_of_sentence(input_text):
  try:
      print(input_text)
      res = get_all_predictions(input_text, top_clean=int(no_words_to_be_predicted)) 
      return res

  except Exception as error:
    pass

**Main execution of the program**

In [19]:
try:
  print("Next Word Prediction with Pytorch using ")
  no_words_to_be_predicted,enter_input_text = set_model_config(no_words_to_be_predicted=3, enter_input_text = "I am looking for")
  gpt_tokenizer, gpt_model  = load_model()
  res = get_prediction_end_of_sentence(enter_input_text)
  print("result is: {}" .format(res))
  answer_gpt = list()
  print(res['gpt'].split("\t"))
  for i in res['gpt'].split("\t"):
      answer_gpt.append(i)
      answer_as_string_gpt = "    ".join(answer_gpt)
      print("predicted statment is: {}" .format(answer_as_string_gpt))

except Exception as e:
  print('problem occured')


Next Word Prediction with Pytorch using 
no_words_to_be_predicted = 3
enter_input_text = I am looking for


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


I am looking for
result is: {'gpt': 'I am looking for a someone people'}
['I am looking for a someone people']
predicted statment is: I am looking for a someone people
